In [6]:
import random
import numpy as np
from prettytable import PrettyTable

In [7]:
'''
valid gene
({A-G}{a-c}{1-3})
'''

subject = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
batch = ['a', 'b', 'c']
room = ['1','2','3']

GENES = [N + n + l for N in subject for n in batch for l in room]

In [8]:
def random_atom(a,b):
    x = random.uniform(0, 1)
    if x <= 0.45:
        return a
    elif x <= 0.9:
        return a
    return random.choice(GENES)

In [37]:
class Gene(object):
    '''
    Class representing individual in population
    '''    
    def __init__(self):
        self.gene = self.create_gene()
        self.decomp = self.decompose(self.gene)
        self.t_s_g_1,self.t_s_g_2,self.t_s_g_3,self.t_c_1,self.t_c_2,self.t_c_3 = self.display(self.gene,0,0,0,0,0,0)
        self.count1,self.count2 = self.get_reward(self.gene,self.t_s_g_1,self.t_s_g_2,self.t_s_g_3,self.t_c_1,self.t_c_2,self.t_c_3)
        self.reward = -self.count1 - self.count2   

    @classmethod
    
    def create_gene(self):
        '''
        create chromosome or string of genes
        Ruby
        '''
        global GENES
        a = ''
        for i in range (0,54) : 
            random_num = random.choice(GENES)
            a = a + str(random_num)
        return a
    
    @classmethod
    def decompose(self,gene):
        arr = np.empty((3,54), str)
        for i in range(0, len(gene),3):
            for j in range(0,3):
                arr[j][i//3] = gene[i+j]
        return arr
    
    @classmethod
    def display(self,gene,a=0,b=0,c=0,d=0,e=0,f=0):
        '''
        Convert the gene code to a user understandable table
        Shambhav
        '''
        #splitting the string into 3 elements each
        n = 3
        new_list = [gene[i:i+n] for i in range(0, len(gene), n)]
        # separate into according to subject group 
        table_student_group_1 = [] 
        table_student_group_2 = [] 
        table_student_group_3 = []

        # according to classes
        table_class_1 = []
        table_class_2 = []
        table_class_3 = []
 
        for j in range(0, len(new_list)):
            if (new_list[j][1]== 'a'):
                table_student_group_1.append(new_list[j])
            elif (new_list[j][1]== 'b'):
                table_student_group_2.append(new_list[j])
            elif (new_list[j][1]== 'c'):
                table_student_group_3.append(new_list[j])

        for j in range(0, len(new_list)):
            if (new_list[j][2]== '1'):
                table_class_1.append(new_list[j])
            elif (new_list[j][2]== '2'):
                table_class_2.append(new_list[j])
            elif (new_list[j][2]== '3'):
                table_class_3.append(new_list[j])
        if a == True:
            print(table_student_group_1)
        if b == True:
            print(table_student_group_2)
        if c == True:
            print(table_student_group_3)
        if d == True:
            print(table_class_1)
        if e == True:
            print(table_class_2)
        if f == True:
            print(table_class_3)
        return table_student_group_1,table_student_group_2,table_student_group_3,table_class_1,table_class_2,table_class_3
    
    @classmethod
    def custom_gene(gene):
        self.gene = gene
    @classmethod
    def get_reward(self,gene,table_student_group_1,table_student_group_2,table_student_group_3,table_class_1,table_class_2,table_class_3,test = False):
        
        n = length_of_atom = 3

        primary_conflict = 0
        secondary_conflict = 0

        new_list = [gene[i:i+n] for i in range(0, len(gene), n)]

        # chromosome is the 3 atoms that represent classes at a certain time slot 
        # i.e classes in sunday 9-11 time slot
        chromosomes = []

        # tables 
        for j in range(0, len(new_list)):
            if (new_list[j][1]== 'a'):
                table_student_group_1.append(new_list[j])
            elif (new_list[j][1]== 'b'):
                table_student_group_2.append(new_list[j])
            elif (new_list[j][1]== 'c'):
                table_student_group_3.append(new_list[j])

        for j in range(0, len(new_list)):
            if (new_list[j][2]== '1'):
                table_class_1.append(new_list[j])
            elif (new_list[j][2]== '2'):
                table_class_2.append(new_list[j])
            elif (new_list[j][2]== '3'):
                table_class_3.append(new_list[j])



        # separate into chromosomes ie. 3 atoms representing classes at a time

        for i in range(0, len(gene), 9):
            chromosome = gene[i:i+9]
            chromosomes.append(chromosome)
            group_count = {combo: 0 for combo in 'abc'}
            class_count = {combo: 0 for combo in '123'}
            # check if two classes at the same classroom
            # and check if two subject group at same time
            #print(chromosome)
            for a in range(0,3):
                if chromosome[2+3*a] in '123':
                    class_count[chromosome[2+3*a]] += 1
                if chromosome[1+3*a] in 'abc':
                    group_count[chromosome[1+3*a]] += 1

            # main checking part
            if  (max([a for a in group_count.values()])>1):
                primary_conflict+=1

                if test == True:
                    # for testing only
                    print(f"group conflict in chromosone {i/9}")

            if  (max([a for a in class_count.values()])>1):
                primary_conflict+=1
                if test == True:
                    # for testing only
                    print(f"class conflict in chromosone {i/9}")

        # to take into account more or less than 18 classes of a subject group 
        if table_student_group_1.count != 18:primary_conflict += 1
        if table_student_group_2.count != 18:primary_conflict += 1
        if table_student_group_3.count != 18:primary_conflict += 1

        ###########################################################
        # For Secondary Conflicts

        # Once subject more than twice or less then once per week
        # Check student group table individually, if subject represented by "ABCDEF"
        # is not repeated exactly twice, then conflict occurs
        # count conflict for each repeated occurance of subject and for each student group table    

        # For student_group_a
        subject_repetitions_count = {combo: 0 for combo in 'ABCDEF'}
        for i in table_student_group_1:
            if i[0] in subject_repetitions_count:
                subject_repetitions_count[i[0]] += 1

        for subject_repetitions_key in subject_repetitions_count.keys():
            if subject_repetitions_count[subject_repetitions_key]!=2:
                secondary_conflict += 1 

                if test == True:
                    # for testing only
                    print(f"subject {subject_repetitions_key} is repeated {subject_repetitions_count[subject_repetitions_key]} times")
                    
        # For student_group_b
        subject_repetitions_count = {combo: 0 for combo in 'ABCDEF'}
        for i in table_student_group_2:
            if i[0] in subject_repetitions_count:
                subject_repetitions_count[i[0]] += 1

        for subject_repetitions_key in subject_repetitions_count.keys():
            if subject_repetitions_count[subject_repetitions_key]!=2:
                secondary_conflict += 1 

                if test == True:
                    # for testing only
                    print(f"subject {subject_repetitions_key} is repeated {subject_repetitions_count[subject_repetitions_key]} times")
        
       # For student_group_c
        subject_repetitions_count = {combo: 0 for combo in 'ABCDEF'}
        for i in table_student_group_3:
            if i[0] in subject_repetitions_count:
                subject_repetitions_count[i[0]] += 1

        for subject_repetitions_key in subject_repetitions_count.keys():
            if subject_repetitions_count[subject_repetitions_key]!=2:
                secondary_conflict += 1 

                if test == True:
                    # for testing only
                    print(f"subject {subject_repetitions_key} is repeated {subject_repetitions_count[subject_repetitions_key]} times")
           
        return primary_conflict,secondary_conflict

In [38]:
g1 = Gene()

In [39]:
g1.gene

'Fa1Fb3Gb3Aa1Fb2Aa3Aa3Ec3Ga1Ac3Ca3Gb2Cc3Da1Aa2Aa3Ec3Da3Cc2Ga1Ab1Ab2Aa1Ca2Gc2Ac2Ac1Gc1Fb1Dc3Ab1Ec1Ec1Cb2Da1Db2Ec2Eb3Ca2Dc3Cc3Gc3Cc3Ga2Eb1Dc1Ac1Da2Ba1Ga3Ga3Aa1Gb1Cc3'

In [40]:
g1.decomp

array([['F', 'F', 'G', 'A', 'F', 'A', 'A', 'E', 'G', 'A', 'C', 'G', 'C',
        'D', 'A', 'A', 'E', 'D', 'C', 'G', 'A', 'A', 'A', 'C', 'G', 'A',
        'A', 'G', 'F', 'D', 'A', 'E', 'E', 'C', 'D', 'D', 'E', 'E', 'C',
        'D', 'C', 'G', 'C', 'G', 'E', 'D', 'A', 'D', 'B', 'G', 'G', 'A',
        'G', 'C'],
       ['a', 'b', 'b', 'a', 'b', 'a', 'a', 'c', 'a', 'c', 'a', 'b', 'c',
        'a', 'a', 'a', 'c', 'a', 'c', 'a', 'b', 'b', 'a', 'a', 'c', 'c',
        'c', 'c', 'b', 'c', 'b', 'c', 'c', 'b', 'a', 'b', 'c', 'b', 'a',
        'c', 'c', 'c', 'c', 'a', 'b', 'c', 'c', 'a', 'a', 'a', 'a', 'a',
        'b', 'c'],
       ['1', '3', '3', '1', '2', '3', '3', '3', '1', '3', '3', '2', '3',
        '1', '2', '3', '3', '3', '2', '1', '1', '2', '1', '2', '2', '2',
        '1', '1', '1', '3', '1', '1', '1', '2', '1', '2', '2', '3', '2',
        '3', '3', '3', '3', '2', '1', '1', '1', '2', '1', '3', '3', '1',
        '1', '3']], dtype='<U1')

In [41]:
g1.reward

-45